<a href="https://colab.research.google.com/github/gabrielfernandorey/ITBA-NLP/blob/main/ITBA_nlp01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabajo Practico NLP - Detección de Tópicos y clasificación
- ITBA 2024
- Alumno: Gabriel Rey
---

## OnLine models

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
pd.set_option('display.max_colwidth', None)
import numpy as np
from matplotlib import pyplot as plt
import os
import json
from datetime import datetime, date
from dateutil.parser import parse
from dotenv import load_dotenv

from NLP_tools import Cleaning_text, top_keywords, top_entities, get_topic_name, best_document, clean_all, topic_documents
from core.functions import *

In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import MiniBatchKMeans
from sklearn.decomposition import IncrementalPCA

from tqdm import tqdm

from umap import UMAP
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.vectorizers import OnlineCountVectorizer

### Path

In [3]:
load_dotenv()
PATH_REMOTO='/content/ITBA-NLP/data/'
PATH=os.environ.get('PATH_LOCAL', PATH_REMOTO)
PATH

'C:/Users/gabri/OneDrive/Machine Learning/Github/ITBA-NLP/data/'

In [4]:
if PATH == os.environ.get('PATH_LOCAL'):
    client = OpenAI(api_key= os.environ.get('OPENAI_API_KEY'))
else:
    from google.colab import userdata
    client = OpenAI(api_key= userdata.get('OPENAI_API_KEY'))

### Cargamos noticias de dos chunks

In [44]:
# Set de datos

batch_news = 500

chunk_1 = '20240715'
df_parquet_1 = pd.read_parquet(PATH+chunk_1+".parquet")
df_parquet_1 = df_parquet_1.sample(n=int(batch_news)).copy()
data_1 = list(df_parquet_1['text'])

chunk_2 = '20240716'
df_parquet_2 = pd.read_parquet(PATH+chunk_2+".parquet")
df_parquet_2 = df_parquet_2.sample(n=int(batch_news)).copy()
data_2 = list(df_parquet_2['text'])

In [45]:
# Unificamos datos
df_parquet = pd.concat([df_parquet_1, df_parquet_2], ignore_index=True)

In [46]:
len(df_parquet)

1000

In [8]:
# Stopwords
SPANISH_STOPWORDS = list(pd.read_csv(PATH+'spanish_stop_words.csv' )['stopwords'].values)

### Modelo 

In [47]:
# Step 1 - Extract embeddings
embedding_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

# Prepare sub-models that support online learning
umap_model = IncrementalPCA(n_components=10)
cluster_model = MiniBatchKMeans(n_clusters=100, random_state=0)
vectorizer_model = OnlineCountVectorizer(stop_words=SPANISH_STOPWORDS, decay=.01)


topic_model = BERTopic(
    embedding_model=embedding_model,
    umap_model=umap_model,
    hdbscan_model=cluster_model,
    vectorizer_model=vectorizer_model,
    ) #calculate_probabilities=True

In [48]:
# Entrenar con el primer set de datos
topic_model.partial_fit(data_1)

In [15]:
def topic_data_view(model, df, probs=None):
    reg = []
    for topic_number in range(len(model.get_topic_info())):
        docs_per_topics = [i for i, x in enumerate(model.topics_) if x == topic_number]
        for doc in docs_per_topics:
            if probs is None:
                reg.append([str(topic_number), str(doc), df.iloc[doc].title])
                columns=['Topic','Id Doc','Title']
            else:
                prob = np.max(probs[doc])
                reg.append([str(topic_number), str(doc), df.iloc[doc].title, prob])
                columns=['Topic','Id Doc','Title','Probs']

    df_query = pd.DataFrame(reg, columns=columns)
    return df_query

In [49]:
df_out_1 = topic_data_view(topic_model, df_parquet_1)
df_out_1

,Topic,Id Doc,Title
0,0,146,El equipo de TN fue echado brutalmente del estadio de la final de la Copa América
1,0,176,La paridad de género llega a los Juegos Olímpicos
2,0,228,Imposible no llorar: los mejores videos de Ángel Di María en la Selección
3,0,270,El penal que no fue y el gol anulado: las polémicas del Argentina-Colombia en la final de la Copa América
4,0,274,Del Obelisco a un paseo en canoa por Tucumán: los festejos en toda Argentina por la conquista de la Copa América
...,...,...,...
495,99,153,Homenaje memorable de la Selección a Ángel Di María en su despedida
496,99,167,Histórico: Argentina venció 1-0 a Colombia en el tiempo extra y se consagró bicampeón de la Copa América
497,99,275,"La última de Di María, el héroe de las mil finales | Adiós grandioso del crack que se hizo ídolo a fuerza de perseverancia y golazos"
498,99,339,Así cantó Abel Pintos el Himno de Argentina en la final


In [ ]:
# Copia del modelo original
model_1 = topic_model

# Realizar un fit_transform en lugar de partial_fit, (para obtener mas adelante las probs)
topics_1 = model_1.fit_transform(data_1)

In [ ]:
def calculate_probabilities(model, docs):
    # Transformar documentos al espacio vectorial
    embeddings = model.embedding_model.embed(docs)

    # Aplicar reducción de dimensionalidad
    reduced_embeddings = model.umap_model.transform(embeddings)
    
    # Predecir clústeres y calcular distancias
    clusters = model.hdbscan_model.predict(reduced_embeddings)
    distances = model.hdbscan_model.transform(reduced_embeddings)
    
    # Aproximar probabilidades (probabilidad inversamente proporcional a la distancia)
    max_distances = np.max(distances, axis=1, keepdims=True)
    probs = 1 - (distances / max_distances)
    return clusters, probs

In [ ]:
# Obtener probabilidades para un nuevo conjunto de datos
topics_1x, probs_1 = calculate_probabilities(model_1, data_1)

In [ ]:
topic_data_view(model_1, df_parquet_1, probs_1)

In [ ]:
def topic_count(topic_model):
    # Obtener documentos de cada tópico
    topic_freq = topic_model.get_topic_freq()

    # Imprimir el número de tópicos encontrados (incluyendo el tópico -1)
    num_topics = len(topic_freq)
    print(f"Número de tópicos encontrados: {num_topics} (incluye el topico -1)")

    # Imprimir la cant de documentos de cada tópico
    print(topic_freq)

In [ ]:
topic_count(model_1)

In [ ]:
topic_labels = model_1.generate_topic_labels()
topic_labels

Resultado parcial set 2 (incremental)

In [13]:
# Entrenar con el segundo set de datos
topic_model.partial_fit(data_2)

In [ ]:
# Copia del modelo original
model_2 = topic_model

# Realizar un fit_transform en lugar de partial_fit, (para obtener mas adelante las probs)
topics_2 = model_2.fit_transform(data_2)

In [ ]:
topic_data_view(model_2, df_parquet_2)

In [16]:
df_iter_2 = topic_data_view(topic_model, df_parquet_2)
df_iter_2

,Topic,Id Doc,Title
0,0,38,La Municipalidad anuncia los cortes de calle por obras para este martes
1,0,64,"Pistas seguras para lograr identificar y detener al asaltante de ""Tiso"" Dam y su esposa"
2,0,75,Nuevos focos de fuego complican el control del incendio en el predio de OSSE
3,0,76,Incendio afectó casi 40 hectáreas del predio de OSSE en ruta 11: controlado y sin viviendas en riesgo
4,0,80,El Gobierno envió al Congreso el proyecto para bajar la edad de imputabilidad a los 13 años
...,...,...,...
995,49,865,Claudio Sesín: acecho e intuición del infinito
996,49,904,Fiesta de la Abuela: «La Peña de Demi» tendrá como figuras a Raly Barrionuevo y Orellana-Lucca
997,49,916,Tras un enigma llamado Guillermo E. Hudson
998,49,917,Después de un año se estrenó Survivor: Expedición Robinson y generó todo tipo de memes


In [ ]:
# Obtener documentos de cada tópico
topic_freq = topic_model.get_topic_freq()

# Imprimir el número de tópicos encontrados (incluyendo el tópico -1)
num_topics = len(topic_freq)
print(f"Número de tópicos encontrados: {num_topics} (incluye el topico -1)")

# Imprimir la cant de documentos de cada tópico
print(topic_freq)

In [ ]:
topic_labels = topic_model.generate_topic_labels()
topic_labels

### Resultados

In [ ]:
# Obtener los n documentos de un tópico
topic_id = 7
n_docs = 5
topic_docs_idx = [i for i, (doc, topic) in enumerate(zip(list(df_parquet_1['title']), topics)) if topic == topic_id]
n_docs = n_docs if n_docs <= len(topic_docs_idx) else len(topic_docs_idx)

print(f"{n_docs} de {len(topic_docs_idx)} titulos de noticias encontrados en el tópico {topic_id:} | {topic_labels[topic_id-1]}")
for idx in topic_docs_idx[:n_docs]:
    print("- ",df_parquet_1.iloc[idx]['title'])